In [1]:
from common_scraping import sleep_r, full_driver, csv_dir_common
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import pickle
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import numpy as np

In [74]:
#driver = full_driver()

### Unfortunately, full_driver() has not worked so I used my own code to run driver.
driver_loc = '/Users/dawidsiwicki/Desktop/TEXT MINING/chromedriver' # place your chromedriver here
driver = webdriver.Chrome(driver_loc, chrome_options=Options())

driver.get('https://www.zdnet.com/topic/')

In [75]:
def topics_links(categories):
    
    pages = driver.find_elements_by_xpath('//ul[@class = "nav nav-tabs topic-nav"]/li//a')
    pages_href = []
    for page in pages:
        pages_href.append(page.get_attribute('href'))
    

    topics_href = []
    for page in pages_href:

        x = True

        driver.get(page)
        
        while x == True:
            
            topics_a = driver.find_elements_by_xpath('//div[@class="col-2"]/h3/a')
            for topic in topics_a:
                if topic.text in categories:
                    topics_href.append(topic.get_attribute('href'))            
            
            try:
                x = True
                next_page = driver.find_element_by_xpath('//a[@class = "next"]')
                driver.get(next_page.get_attribute('href'))
            except:
                x = False
                continue
      
    
    topics_href = np.unique(topics_href)
    return topics_href




In [76]:
def articles_links (time_border, topics, arts):


    for topic in topics:
        
        first_date =pd.to_datetime('now') 
        i = 1
        while first_date > time_border:

            driver.get(topic + '/' + str(i))
            sleep_r('m')
            try:
                
                first_date = pd.to_datetime(driver.find_element_by_xpath('//article[@class = "item"]/div/div/p/span').get_attribute('data-date'))
            except:
                continue
            
            i = i + 1
            
            #category = driver.find_element_by_xpath('//header/h1[@itemprop = "headline"]').text
            category = topics[0].split('/')[-2]
            sleep_r('m')
            
            articles_all = driver.find_elements_by_xpath('//section[@id="topic-river-latest"]/div/div/div/article')
            if not len(articles_all):
                break
            elif pd.to_datetime(driver.find_element_by_xpath('//article[@class = "item"]/div/div/p/span').get_attribute('data-date')) < time_border:
                break
   
            else:
                for article in articles_all:

                    article_link = article.find_element_by_xpath('.//h3/a').get_attribute('href')
                    article_title = article.find_element_by_xpath('.//h3/a').text  # 2a
                    article_abstract = article.find_element_by_xpath('.//p[@class = "summary"]').text  # 2b
                    
                    try:
                        article_author = article.find_element_by_xpath('.//p[@class="meta"]/a').text  # 2c
                    except NoSuchElementException:
                        article_author = ''
                        
                    try:
                        date = article.find_element_by_xpath('.//p[@class="meta"]/span')
                        article_date = pd.to_datetime(date.get_attribute('data-date'))  # 2d
                    except NoSuchElementException:
                        article_date = ''
                    
                    arts.append({'link': article_link,
                                 'category': category,
                                 'title_outside': article_title,  # 2a
                                 'abstract_outside': article_abstract  # 2b
                                  ,'author_outside': article_author,  # 2c
                                  'date_outside': article_date  # 2d
                                    }  
                                )
                    np.save('arts.npy', arts)
                    sleep_r('m')
                    
                    
    arts = [row for row in arts if pd.to_datetime(row['date_outside']) > time_border]
    return arts




In [77]:
def download_articles(all_articles, csv_dir):
    for i_art, art in enumerate(all_articles):
        link_dict = {}
        print(i_art, len(all_articles), art['link'])
        
        try:
            driver.get(art['link'])

            sleep_r('m')
            link_dict['link'] = art['link']

            try:
                link_dict['title'] = driver.find_element_by_xpath('//header/h1[@itemprop = "headline"]').text

            except NoSuchElementException:
                link_dict['title'] = ''


            try:
                link_dict['author'] = driver.find_element_by_xpath('//p/a[@itemprop="author"]').text
            except NoSuchElementException:
                link_dict['author'] = ''

            try:
                link_dict['description'] = driver.find_element_by_xpath('//header[@class="storyHeader article"]/p[@itemprop="description alternativeHeadline"]').text
            except NoSuchElementException:
                link_dict['description'] = ''

            try:
                link_dict['date'] = driver.find_element_by_xpath('//header[@class="storyHeader article"]/div/p/time').get_attribute('datetime')  # 5a
            except NoSuchElementException:
                continue

            article_p_list = []
            for p in [x.text for x in driver.find_elements_by_xpath('//article/div/p')]:
                article_p_list.append(p)

            link_dict['text'] = '\n\n'.join(article_p_list)    

            art.update(link_dict)
            all_articles[i_art] = art
 
         
        except TimeoutException:
            print('timeout', art['link'])
            all_articles.append(art)
            continue
            
    pd.DataFrame(all_articles).to_pickle('zdnet.pkl')
    pd.DataFrame(all_articles).to_csv('zdnet.csv')
    #return(all_articles)




In [78]:
topics_list = ['Security']

In [79]:
time_limit = pd.to_datetime('now') - pd.Timedelta('365 days')
csvs = csv_dir_common()

In [80]:
topics = topics_links(categories = topics_list)

In [85]:
articles = []
articles = articles_links(time_border = time_limit, topics = topics, arts = articles)

#articles_df = pd.DataFrame(articles)

In [57]:
### Additional version to check 

# topics2 = ['https://www.zdnet.com/topic/productivity/']

# articles = []
# articles = articles_links(time_border = time_limit, topics = topics, arts = articles)

In [88]:
pd.DataFrame(articles).to_csv('articles_out.csv')

In [89]:
download_articles(all_articles = articles, csv_dir = csvs)

0 2262 https://www.zdnet.com/article/the-real-future-of-healthcare-is-cultural-change-not-just-ai-and-other-technology/
1 2262 https://www.zdnet.com/article/rowhammer-attacks-can-now-bypass-ecc-memory-protections/
2 2262 https://www.zdnet.com/article/sim-swapping-21-year-old-scores-1-million-by-hijacking-a-phone/
3 2262 https://www.zdnet.com/article/usps-finally-fixes-website-flaw-that-exposed-60-million-users-data/
4 2262 https://www.zdnet.com/article/how-the-dropbox-red-team-found-an-apple-zero-day-exploit-chain-during-pen-tests/
5 2262 https://www.zdnet.com/article/facebook-appeals-500000-penalty-over-cambridge-analytica-scandal/
6 2262 https://www.zdnet.com/article/emotet-malware-runs-on-a-dual-infrastructure-to-avoid-downtime-and-takedowns/
7 2262 https://www.zdnet.com/article/australian-pm-insists-on-encryption-busting-bill-being-passed-in-next-sitting-fortnight/
8 2262 https://www.zdnet.com/article/new-entrepreneur-visa-rolled-out-in-south-australia/
9 2262 https://www.zdnet.com

77 2262 https://www.zdnet.com/article/why-wannacry-ransomware-is-still-a-threat-to-your-pc/
78 2262 https://www.zdnet.com/article/cisco-updates-sd-wan-portfolio-with-new-security-features/
79 2262 https://www.zdnet.com/article/google-traffic-hijacked-via-tiny-nigerian-isp/
80 2262 https://www.zdnet.com/article/how-magecart-groups-are-stealing-your-card-details-from-online-stores/
81 2262 https://www.zdnet.com/article/iot-security-why-everyone-needs-to-step-to-ensure-the-security-of-the-internet-of-things/
82 2262 https://www.zdnet.com/article/how-to-make-use-of-serverless/
83 2262 https://www.zdnet.com/article/data61-and-cba-demonstrate-blockchain-welfare-payments/
84 2262 https://www.zdnet.com/article/local-councils-and-taxi-commission-continued-to-seek-telco-metadata-comms-alliance/
85 2262 https://www.zdnet.com/article/google-launches-visbug-a-chrome-extension-for-point-and-click-web-design/
86 2262 https://www.zdnet.com/article/us-russia-china-dont-sign-macrons-cyber-pact/
87 2262 

155 2262 https://www.zdnet.com/article/australian-shipbuilder-defense-contractor-reveals-data-breach-extortion-demands/
156 2262 https://www.zdnet.com/article/giant-ransomware-bundle-threatens-to-make-malware-attacks-easier-for-crooks/
157 2262 https://www.zdnet.com/article/fifa-admits-hack-and-braces-for-new-leaks/
158 2262 https://www.zdnet.com/article/us-senator-working-on-bill-that-would-jail-ceos-for-user-privacy-violations/
159 2262 https://www.zdnet.com/article/australian-defence-contractor-austral-hit-by-data-breach/
160 2262 https://www.zdnet.com/article/us-charges-china-taiwan-firms-with-stealing-microns-dram-technology/
161 2262 https://www.zdnet.com/article/almost-half-of-usb-drives-in-industrial-settings-pose-severe-security-risk/
162 2262 https://www.zdnet.com/article/major-windows-10-office-365-upgrade-for-more-than-100000-nhs-staff/
163 2262 https://www.zdnet.com/article/cia-vault7-leaker-to-be-charged-for-leaking-more-classified-data-while-in-prison/
164 2262 https://w

233 2262 https://www.zdnet.com/article/phishing-attacks-why-is-email-still-such-an-easy-target-for-hackers/
234 2262 https://www.zdnet.com/article/yahoo-agrees-to-pay-50-million-to-settle-massive-data-breach/
235 2262 https://www.zdnet.com/article/my-health-record-opt-outs-now-sit-at-over-1-1-million/
236 2262 https://www.zdnet.com/article/culture-the-missing-link-for-cybersecuritys-weakest-link/
237 2262 https://www.zdnet.com/article/wells-fargo-banks-entering-a-legal-quagmire-in-the-name-of-technology/
238 2262 https://www.zdnet.com/article/microsoft-windows-zero-day-disclosed-on-twitter-again/
239 2262 https://www.zdnet.com/article/firefox-63-released-with-always-on-tracking-protection/
240 2262 https://www.zdnet.com/article/magecart-group-leverages-zero-days-in-20-magento-extensions/
241 2262 https://www.zdnet.com/article/fireeye-links-russian-research-lab-to-triton-ics-malware-attacks/
242 2262 https://www.zdnet.com/article/this-is-how-promethium-malware-operators-used-security-re

309 2262 https://www.zdnet.com/article/ibm-launches-ibm-ai-openscale-to-combat-ai-sprawl-as-part-of-broad-open-strategy/
310 2262 https://www.zdnet.com/article/chrome-edge-ie-firefox-and-safari-to-disable-tls-1-0-and-tls-1-1-in-2020/
311 2262 https://www.zdnet.com/article/octopus-trojan-exploits-telegram-ban-to-snag-targets-across-asia/
312 2262 https://www.zdnet.com/article/sony-working-on-a-fix-for-bug-thats-crashing-playstation-4-consoles/
313 2262 https://www.zdnet.com/article/apple-voiceover-iphone-vulnerability-permits-access-to-user-photos/
314 2262 https://www.zdnet.com/article/iot-security-follow-these-rules-to-protect-your-users-from-hackers-gadget-makers-told/
315 2262 https://www.zdnet.com/article/my-health-record-privacy-amendments-woefully-inadequate-labor/
316 2262 https://www.zdnet.com/article/were-killing-off-passwords-but-are-we-ready-for-what-will-replace-them/
317 2262 https://www.zdnet.com/article/australian-encryption-busting-bill-would-create-backdoors-cisco/
318

...

1930 2262 https://www.zdnet.com/article/security-consultant-granted-bail-after-hacking-goget-systems/
1931 2262 https://www.zdnet.com/article/australian-government-cannot-handle-its-own-data-securely-why-give-it-yours/
1932 2262 https://www.zdnet.com/article/intel-names-new-technology-chief-amid-meltdown-spectre-fallout/
1933 2262 https://www.zdnet.com/article/firefox-security-mozilla-issues-fix-for-critical-html-hijack-flaw/
1934 2262 https://www.zdnet.com/article/it-budgets-in-brazil-to-increase-in-2018/
1935 2262 https://www.zdnet.com/article/a-year-later-camera-makers-still-resist-encryption/
1936 2262 https://www.zdnet.com/article/cryptocurrency-miners-a-replacement-for-ransomware/
1937 2262 https://www.zdnet.com/article/in-fingerprints-and-banks-we-trust-ibm-reports-on-the-future-of-authentication/
1938 2262 https://www.zdnet.com/article/hackable-ct-mri-machines-come-under-fire-in-new-research/
1939 2262 https://www.zdnet.com/article/amd-vs-spectre-our-new-zen-2-chips-will-be-pro

2009 2262 https://www.zdnet.com/article/triton-exploited-zero-day-flaw-to-target-industrial-systems/
2010 2262 https://www.zdnet.com/article/dridex-banking-trojan-compromises-ftp-sites-in-new-campaign/
2011 2262 https://www.zdnet.com/article/notifiable-data-breaches-scheme-getting-ready-to-disclose-a-data-breach-in-australia/
2012 2262 https://www.zdnet.com/article/muscat-securities-market-was-easily-hackable-for-months/
2013 2262 https://www.zdnet.com/article/windows-10-meltdown-spectre-patch-new-updates-bring-fix-for-unbootable-amd-pcs/
2014 2262 https://www.zdnet.com/article/this-android-malware-wants-to-steal-your-facebook-login-and-bombard-you-with-ads/
2015 2262 https://www.zdnet.com/article/meltdown-spectre-intel-says-newer-chips-also-hit-by-unwanted-reboots-after-patch/
2016 2262 https://www.zdnet.com/article/google-awards-researcher-over-110000-for-android-exploit-chain/
2017 2262 https://www.zdnet.com/article/watchguard-snaps-up-dns-security-firm-percipient-networks/
2018 226

2087 2262 https://www.zdnet.com/article/australian-government-releases-small-business-cybersecurity-guide/
2088 2262 https://www.zdnet.com/article/privacy-foundation-trusting-government-with-open-data-a-recipe-for-pain/
2089 2262 https://www.zdnet.com/article/singapore-reviews-move-to-introduce-legislation-against-fake-news/
2090 2262 https://www.zdnet.com/article/verizon-acquires-niddel/
2091 2262 https://www.zdnet.com/article/how-the-meltdown-and-spectre-security-holes-fixes-will-affect-you/
2092 2262 https://www.zdnet.com/article/warrantless-phone-laptop-searches-at-the-us-border-hit-record-levels/
2093 2262 https://www.zdnet.com/article/amazon-turns-over-record-amount-of-customer-data-to-us-law-enforcement/
2094 2262 https://www.zdnet.com/article/zero-day-vulnerabilities-hijack-full-dell-emc-data-protection-suite/
2095 2262 https://www.zdnet.com/article/android-security-flashlight-apps-on-google-play-infested-with-adware-were-downloaded-by-1-5m-people/
2096 2262 https://www.zdnet.c

2164 2262 https://www.zdnet.com/article/security-researcher-says-directv-hardware-can-be-easily-hacked/
2165 2262 https://www.zdnet.com/article/the-most-high-profile-cryptocurrency-data-breaches-vulnerabilities-and-disasters-in-2017/
2166 2262 https://www.zdnet.com/article/fcc-revisited-net-neutrality-changes-are-misleading-and-not-benign-says-gewirtz/
2167 2262 https://www.zdnet.com/article/justice-dept-indicts-mirai-botnet-attackers/
2168 2262 https://www.zdnet.com/article/singapore-defence-ministry-invites-hackers-to-breach-its-systems/
2169 2262 https://www.zdnet.com/article/robot-exploit-from-1998-resurrected-leaves-top-sites-crypto-vulnerable/
2170 2262 https://www.zdnet.com/article/almost-one-billion-video-stream-users-exposed-to-secret-cryptocurrency-mining/
2171 2262 https://www.zdnet.com/article/adobe-patches-business-logic-error-in-flash/
2172 2262 https://www.zdnet.com/article/individualism-may-make-you-better-at-catching-a-phish-research/
2173 2262 https://www.zdnet.com/ar

2242 2262 https://www.zdnet.com/article/snoopers-beware-googles-ai-can-now-spot-shoulder-surfers-peeking-at-your-screen/
2243 2262 https://www.zdnet.com/article/internet-censorship-its-on-the-rise-and-silicon-valley-is-helping-it-happen/
2244 2262 https://www.zdnet.com/article/us-indicts-chinese-hackers-for-corporate-espionage/
2245 2262 https://www.zdnet.com/article/ombudsman-finds-australian-federal-police-unaware-of-journalist-metadata-requirements/
2246 2262 https://www.zdnet.com/article/trump-considering-banning-white-house-staffers-personal-phones/
2247 2262 https://www.zdnet.com/article/after-wannacry-ransomware-attack-the-nhs-is-toughening-its-cyber-defences/
2248 2262 https://www.zdnet.com/article/mcafee-buys-skyhigh-networks-to-bolster-cloud-business/
2249 2262 https://www.zdnet.com/article/department-of-social-services-says-it-has-contained-data-breach-vulnerability/
2250 2262 https://www.zdnet.com/article/australians-will-trade-privacy-for-security-if-you-frame-it-right/
22

In [16]:
texts = pd.read_csv('csv_08/zdnet.csv')

In [17]:
pd.DataFrame(texts)

,Unnamed: 0,abstract_outside,author,author_outside,category,date,date_outside,description,link,text,title,title_outside
0,0,Governments need to create digital health poli...,Stilgherrian,Stilgherrian,security,2018-11-23 05:31:00,2018-11-23 05:31:00,Governments need to create digital health poli...,https://www.zdnet.com/article/the-real-future-...,"""It actually is quite easy to be a futurist wi...",The real future of healthcare is cultural chan...,The real future of healthcare is cultural chan...
1,1,Attack works against ECC memory included with ...,Catalin Cimpanu,Catalin Cimpanu,security,2018-11-22 15:11:17,2018-11-22 15:11:17,Attack works against ECC memory included with ...,https://www.zdnet.com/article/rowhammer-attack...,Academics from the Vrije University in Amsterd...,Rowhammer attacks can now bypass ECC memory pr...,Rowhammer attacks can now bypass ECC memory pr...
2,2,The man reportedly targeted well-known busines...,Charlie Osborne,Charlie Osborne,security,2018-11-22 12:54:32,2018-11-22 12:54:32,The man reportedly targeted well-known busines...,https://www.zdnet.com/article/sim-swapping-21-...,A 21-year-old has been accused of SIM-swapping...,SIM-swapping 21-year-old scores $1 million by ...,SIM-swapping 21-year-old scores $1 million by ...
3,3,US Postal Service website flaw was patched thi...,Liam Tung,Liam Tung,security,2018-11-22 12:13:00,2018-11-22 12:13:00,US Postal Service website flaw was patched thi...,https://www.zdnet.com/article/usps-finally-fix...,The US Postal Service has fixed a security bug...,USPS finally fixes website flaw that exposed 6...,USPS finally fixes website flaw that exposed 6...
4,4,The zero-day vulnerabilities were accidentally...,Charlie Osborne,Charlie Osborne,security,2018-11-22 10:54:20,2018-11-22 10:54:20,The zero-day vulnerabilities were accidentally...,https://www.zdnet.com/article/how-the-dropbox-...,While probing Dropbox and how the cloud storag...,How Dropbox's red team discovered an Apple zer...,How Dropbox's red team discovered an Apple zer...
5,5,"The fine, imposed by the ICO, was the maximum ...",Charlie Osborne,Charlie Osborne,security,2018-11-22 09:05:00,2018-11-22 09:05:00,"The fine, imposed by the ICO, was the maximum ...",https://www.zdnet.com/article/facebook-appeals...,"Facebook has appealed a ruling and £500,000 fi...","Facebook appeals £500,000 penalty over Cambrid...","Facebook appeals £500,000 penalty over Cambrid..."
6,6,Researchers spot unique design in the server i...,Catalin Cimpanu,Catalin Cimpanu,security,2018-11-22 06:00:03,2018-11-22 06:00:03,Researchers spot unique design in the server i...,https://www.zdnet.com/article/emotet-malware-r...,The Emotet malware gang is probably managing t...,Emotet malware runs on a dual infrastructure t...,Emotet malware runs on a dual infrastructure t...
7,7,Australian agencies need the power to intercep...,Chris Duckett,Chris Duckett,security,2018-11-22 03:56:00,2018-11-22 03:56:00,Australian agencies need the power to intercep...,https://www.zdnet.com/article/australian-pm-in...,Australian Prime Minister Scott Morrison has c...,Australian PM insists on encryption-busting Bi...,Australian PM insists on encryption-busting Bi...
8,8,A new entrepreneur visa program is being rolle...,Chris Duckett,Chris Duckett,security,2018-11-22 03:08:55,2018-11-22 03:08:55,A new entrepreneur visa program is being rolle...,https://www.zdnet.com/article/new-entrepreneur...,A new pilot visa program will attract budding ...,New entrepreneur visa rolled out in South Aust...,New entrepreneur visa rolled out in South Aust...
9,9,City IT network was infected by Hermes ransomw...,Catalin Cimpanu,Catalin Cimpanu,security,2018-11-21 23:23:00,2018-11-21 23:23:00,City IT network was infected by Hermes ransomw...,https://www.zdnet.com/article/city-of-valdez-a...,"Officials from the city of Valdez, Alaska have...","City of Valdez, Alaska admits to paying off ra...","City of Valdez, Alaska admits to paying off ra..."


In [18]:
pd.to_datetime(texts.date_outside[2261]) - pd.to_datetime(texts.date_outside[0])

Timedelta('-365 days +04:28:02')

In [19]:
pd.to_datetime(texts.date[2261]) - pd.to_datetime(texts.date[0])

Timedelta('-365 days +04:28:02')

In [22]:
len(set(texts.author))

79

In [23]:
len(set(texts.author_outside))

78